# scRNA-seq Analysis of *Parhyale hawaiensis* with Scanpy

Author: Miquel Sendra  
This notebook follows best practices for reproducible single-cell RNA-seq analysis using `scanpy`, starting from STARsolo outputs.

In [ ]:
# Imports and settings
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import os

# Set up paths
DATA_DIR = Path("../scParhyale_data/raw")
METADATA_FILE = Path("../scParhyale_data/HHMI_QG_Sample_Manifest_Form_v1.2_Pavlopoulos_QC.xlsx")
LIBRARIES = ["lib01", "lib02", "lib03", "lib04", "lib05", "lib06"]

## Load Metadata

In [ ]:
# Load metadata
metadata_df = pd.read_excel(METADATA_FILE, usecols="A:B", skiprows=1, nrows=6)
metadata_df.columns = ["Count", "Sample Name"]
metadata_df["Library ID"] = [f"lib0{i}" for i in metadata_df['Count']]
metadata_df["Phenotype"] = metadata_df["Sample Name"].apply(lambda x: "WT" if "Wild-type" in x else "Ablated")
metadata_df["Stage"] = metadata_df["Sample Name"].apply(lambda x: x.split()[-1])
metadata_df.set_index("Library ID", inplace=True)
metadata_df

## Load STARsolo matrices and build combined AnnData object

In [ ]:
# Load libraries and store AnnData objects
adatas = []
for lib in LIBRARIES:
    ad = sc.read_10x_mtx(DATA_DIR / lib, var_names='gene_symbols', cache=True)
    ad.obs["library_id"] = lib
    ad.obs = ad.obs.merge(metadata_df, left_on="library_id", right_index=True)
    ad.var_names_make_unique()
    adatas.append(ad)

# Concatenate all
adata = adatas[0].concatenate(*adatas[1:], batch_key="batch", batch_categories=LIBRARIES)
adata

## Preprocessing

In [ ]:
# Basic filtering
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
# Calculate QC metrics
adata.var["mt"] = adata.var_names.str.startswith("mt-")
sc.pp.calculate_qc_metrics(adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True)
# Plot QC
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], 
             jitter=0.4, multi_panel=True)

## Filtering based on QC

In [ ]:
# Apply thresholds based on inspection
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.pct_counts_mt < 10, :]
adata

## Normalization and Log Transformation

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
adata.raw = adata

## Highly Variable Genes

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)
adata = adata[:, adata.var.highly_variable]

## PCA, Neighbors, UMAP

In [ ]:
sc.pp.scale(adata, max_value=10)
sc.tl.pca(adata, svd_solver='arpack')
sc.pl.pca_variance_ratio(adata, log=True)
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
sc.tl.umap(adata)
sc.pl.umap(adata, color=['library_id', 'Phenotype', 'Stage'])

## Clustering

In [ ]:
sc.tl.leiden(adata, resolution=0.5)
sc.pl.umap(adata, color=['leiden'])

## Marker Gene Identification

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=20, sharey=False)

## Save Processed Data

In [ ]:
adata.write("../scParhyale_data/processed/scParhyale_adata.h5ad")